In [1]:
# !conda create -n crewai python=3.11
# !conda activate crewai -
%pip install -U 'crewai[tools]' mdpdf 
%pip install -U duckduckgo-search
%pip install -U langchain_google_vertexai
!pip install -U langchain-google-vertexai


  Using cached crewai-0.98.0-py3-none-any.whl.metadata (27 kB)
  Using cached blinker-1.9.0-py3-none-any.whl.metadata (1.6 kB)
  Using cached chromadb-0.6.3-py3-none-any.whl.metadata (6.8 kB)
  Using cached litellm-1.57.4-py3-none-any.whl.metadata (36 kB)
  Using cached openpyxl-3.1.5-py2.py3-none-any.whl.metadata (2.5 kB)
  Using cached crewai_tools-0.32.1-py3-none-any.whl.metadata (6.0 kB)
  Using cached chroma_hnswlib-0.7.6-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (252 bytes)
  Using cached linkup_sdk-0.2.2-py3-none-any.whl.metadata (4.2 kB)
  Using cached patronus-0.0.17-py3-none-any.whl.metadata (4.2 kB)
  Using cached scrapegraph_py-1.10.0-py3-none-any.whl.metadata (5.7 kB)
  Using cached serpapi-0.1.5-py2.py3-none-any.whl.metadata (10 kB)
  Using cached snowflake-1.0.2-py3-none-any.whl.metadata (1.9 kB)
  Using cached spider_client-0.1.25-py3-none-any.whl
  Using cached weaviate_client-4.10.4-py3-none-any.whl.metadata (3.6 kB)
  Using cached et_xmlfile

In [8]:
%pip install --upgrade --quiet  langchain-core langchain-google-vertexai
%pip install mdpdf

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import re

PROJECT_ID = !(gcloud config get-value core/project)
PROJECT_ID = PROJECT_ID[0]

LOCATION="asia-southeast1"

FOLDER_NAME="."

#!/usr/bin/env python
# coding: utf-8

# PROJECT_ID = os.environ.get("GOOGLE_CLOUD_PROJECT")
# SVC_ACC = os.environ.get("SERVICE_ACCOUNT")
# PROJECT_NUMBER = str(re.search(r'\d+', SVC_ACC).group()) if SVC_ACC else None



In [12]:
from crewai import Agent, Task, Crew, Process
from crewai_tools.tools import FileReadTool
import os, requests, re, mdpdf, subprocess
import uuid

from langchain_community.vectorstores import Chroma
from langchain_community.tools import DuckDuckGoSearchRun

from typing import Dict

from crewai import Agent, Task, Crew, Process
from crewai_tools.tools import FileReadTool
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_google_vertexai import ChatVertexAI
from langchain.tools import tool
from vertexai.preview.vision_models import ImageGenerationModel
import mdpdf

In [13]:
# Initialize Gemini LLM
llm = ChatVertexAI(
    model_name='gemini-1.5-pro-002',
    project=PROJECT_ID,
    location="us-central1",
)

@tool
def generateimage(chapter_content_and_character_details: Dict) -> str:
    """
    Generates an image using Imagen for a given chapter content and character details and return the image_url.
    """

    if 'description' not in chapter_content_and_character_details:
        raise ValueError("The input dictionary must contain a 'description' key.")

    model = ImageGenerationModel.from_pretrained("imagen-3.0-generate-001")
    description = chapter_content_and_character_details['description']

    prompt = f"Image is about: {description}. Style: Illustration. Create an illustration incorporating a vivid palette with an emphasis on shades of azure and emerald, augmented by splashes of gold for contrast and visual interest. The style should evoke the intricate detail and whimsy of early 20th-century storybook illustrations, blending realism with fantastical elements to create a sense of wonder and enchantment. The composition should be rich in texture, with a soft, luminous lighting that enhances the magical atmosphere. Attention to the interplay of light and shadow will add depth and dimensionality, inviting the viewer to delve into the scene. DON'T include ANY text in this image. DON'T include colour palettes in this image."

    response = model.generate_images(
        prompt=prompt,
        number_of_images=2,
        language="en",
        aspect_ratio="1:1",
        safety_filter_level="block_some",
    )

    image_url_name = str(uuid.uuid4())
    filepath = os.path.join(os.getcwd(), "images1/")
    os.makedirs(filepath, exist_ok=True)

    image_url = os.path.join(filepath, image_url_name + ".jpg")
    response.images[0].save(location=image_url)
    print("url", image_url)

    return image_url

# Instantiate tools
file_read_tool = FileReadTool(
    file_path='template.md',
    description='A tool to read the Story Template file and understand the expected output format.'
)

search_tool = DuckDuckGoSearchRun()


@tool
def convermarkdowntopdf(markdownfile_name: str) -> str:
    """
    Converts a Markdown file to a PDF document using the mdpdf command line application.

    Args:
        markdownfile_name (str): Path to the input Markdown file.

    Returns:
        str: Path to the generated PDF file.
    """
    output_file = os.path.splitext(markdownfile_name)[0] + '.pdf'
    cmd = ['mdpdf', '--output', output_file, markdownfile_name]
    subprocess.run(cmd, check=True)
    return output_file

story_outliner = Agent(
    role='Story Outliner',
    goal='Develop an outline for a children\'s storybook about Animals, including chapter titles and characters for 5 chapters.',
    backstory="An imaginative creator who lays the foundation of captivating stories for children.",
    verbose=True,
    llm=llm,
    allow_delegation=False
)

story_writer = Agent(
    role='Story Writer',
    goal='Write the full content of the story for all 5 chapters, each chapter 300 words, weaving together the narratives and characters outlined.',
    backstory="A talented storyteller who brings to life the world and characters outlined, crafting engaging and imaginative tales for children.",
    verbose=True,
    llm=llm,
    allow_delegation=False
)

image_generator = Agent(
    role='Image Generator',
    goal='Generate one image per chapter content provided by the story outliner. Start with Chapter number, chapter content, character details, detailed location information and detailed items in the location where the activity happens. Generate totally 5 images one by one. Final output should contain all the 5 images in json format.',
    backstory="A creative AI specialized in visual storytelling, bringing each chapter to life through imaginative imagery.",
    verbose=True,
    llm=llm,
    tools=[generateimage],
    allow_delegation=False
)

content_formatter = Agent(
    role='Content Formatter',
    goal='Format the written story content in markdown, including images at the beginning of each chapter.',
    backstory='A meticulous formatter who enhances the readability and presentation of the storybook.',
    verbose=True,
    llm=llm,
    tools=[file_read_tool],
    allow_delegation=False
)

markdown_to_pdf_creator = Agent(
    role='PDF Converter',
    goal='Convert the Markdown file to a PDF document. story.md is the markdown file name.',
    backstory='An efficient converter that transforms Markdown files into professionally formatted PDF documents.',
    verbose=True,
    llm=llm,
    tools=[convermarkdowntopdf],
    allow_delegation=False
)


# Create tasks for the agents
task_outline = Task(
    description='Create an outline for the children\'s storybook about Animals, detailing chapter titles and character descriptions for 5 chapters.',
    agent=story_outliner,
    expected_output='A structured outline document containing 5 chapter titles, with detailed character descriptions and the main plot points for each chapter.'
)

task_write = Task(
    description='Using the outline provided, write the full story content for all chapters, ensuring a cohesive and engaging narrative for children. Each Chapter 100 words. Include Title of the story at the top.',
    agent=story_writer,
    expected_output='A complete manuscript of the children\'s storybook about Animals with 5 chapters. Each chapter should contain approximately 100 words, following the provided outline and integrating the characters and plot points into a cohesive narrative.'
)

task_image_generate = Task(
    description='Generate 5 images that captures the essence of the children\'s storybook about Animals, aligning with the themes, characters, and narrative outlined for the chapters. Do it one by one.',
    agent=image_generator,
    expected_output='A digital image file that visually represents the overarching theme of the children\'s storybook, incorporating elements from the characters and plot as described in the outline. The image should be suitable for inclusion in the storybook as an illustration.',
)

task_format_content = Task(
    description='Format the story content in markdown, including an image at the beginning of each chapter.',
    agent=content_formatter,
    expected_output='The entire storybook content formatted in markdown, with each chapter title followed by the corresponding image and the chapter content.',
    context=[task_write, task_image_generate],
    output_file="story.md"
)

task_markdown_to_pdf = Task(
    description='Convert a Markdown file to a PDF document, ensuring the preservation of formatting, structure, and embedded images using the mdpdf library.',
    agent=markdown_to_pdf_creator,
    expected_output='A PDF file generated from the Markdown input, accurately reflecting the content with proper formatting. The PDF should be ready for sharing or printing.'
)

crew = Crew(
    agents=[story_outliner, story_writer, image_generator, content_formatter, markdown_to_pdf_creator],
    tasks=[task_outline, task_write, task_image_generate, task_format_content, task_markdown_to_pdf],
    verbose=True,
    process=Process.sequential
)

result = crew.kickoff()

print(result)

2025-01-21 04:31:38,766 - 139783677891776 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


LLM value is an unknown object
LLM value is an unknown object
LLM value is an unknown object
LLM value is an unknown object
LLM value is an unknown object
# Agent: Story Outliner
## Task: Create an outline for the children's storybook about Animals, detailing chapter titles and character descriptions for 5 chapters.


# Agent: Story Outliner
## Final Answer: 
**Title:** The Great Forest Feast

**Overall Story:** A group of unlikely animal friends must work together to overcome their differences and retrieve the stolen ingredients for the annual Forest Feast.

**Chapter 1: The Missing Berries**

*   **Title:** The Empty Berry Bush
*   **Characters:**
    *   **Barnaby Bear:** A clumsy but kind-hearted bear with a love for berries.
    *   **Fiona Fox:** A clever and resourceful fox with a knack for problem-solving.
*   **Plot:** Barnaby discovers that all the berries for the Forest Feast have disappeared. He enlists Fiona's help to find out what happened.

**Chapter 2: The Whispering Wi

Logging to /home/jupyter/GenAI7/ai_agents_v2/mdpdf.log
story.md




# Agent: PDF Converter
## Thought: Thought: I need to convert the markdown file `story.md` to a PDF file.  I can use the `convermarkdowntopdf` tool for this.
## Using tool: convermarkdowntopdf
## Tool Input: 
"{\"markdownfile_name\": \"story.md\"}"
## Tool Output: 
story.pdf


# Agent: PDF Converter
## Final Answer: 
story.pdf


story.pdf
